# Поэтапный процесс ввода модели в эксплуатацию

Процесс ввода в эксплуатацию должен снижать риск внесения некорректных изменений в сервис.



Основная задача в данной работе познакомиться с практической частью процесса проверки кандидатов на ввод в эксплуатацию на различных этапах. 



Процесс непрерывной интеграции также можно автоматизировать, чтобы исключить человеческий фактор при тестировании версий моделей.



Все изменения также можно смотреть в [интерфейсе MLflow](/app/)

#### 0. Подготовка данных экспериментов

Импортируем необходимые модули и определеим переменные.

Код аналогичен первой лабораторной работе, заполняем реестр экспериментов для дальнейшей работы с ними.

In [4]:
import os
import sys
import warnings
import pprint

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

MLFLOW_SERVER_URL = 'http://127.0.0.1:5000/'
experiment_name = 'experiment-for-ci'

warnings.filterwarnings("ignore")
np.random.seed(40)
data = pd.read_csv("mlflow-example/wine-quality.csv")

train, test = train_test_split(data)

train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]
# отложенная выборка
test_later_x, test_x = test_x[:10], test_x[10:]
test_later_y, test_y = test_y[:10], test_y[10:]

client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)

# подключаемся к серверу
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

mlflow.set_experiment(experiment_name)

# запуск в эксперименте

for alpha, l1_ratio in ((0.3, 0.5), (0.3, 0.3), (0.8, 0.5), (0.45, 0.3), (0.2, 0.3), (0.9, 0.9)):
    with mlflow.start_run():
        # модель
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # метрики
        predicted_qualities = lr.predict(test_x)
        rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))
        mae = mean_absolute_error(test_y, predicted_qualities)
        r2 = r2_score(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # сохраняем значения эксперимента в системе
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

experiment = client.get_experiment_by_name(experiment_name)
reg_model_name = "sk-learn-model-ci"
client.create_registered_model(reg_model_name)
# staging model
run_info = client.list_run_infos(experiment.experiment_id)[0]
result = client.create_model_version(
    name=reg_model_name,
    source=f"{run_info.artifact_uri}/mdel",
    run_id=run_info.run_id
)
client.transition_model_version_stage(
    name=reg_model_name,
    version=result.version,
    stage="Staging"
)
# prod model
run_info = client.list_run_infos(experiment.experiment_id)[1]
result = client.create_model_version(
    name=reg_model_name,
    source=f"{run_info.artifact_uri}/model",
    run_id=run_info.run_id
)
client.transition_model_version_stage(
    name=reg_model_name,
    version=result.version,
    stage="Production"
)

Elasticnet model (alpha=0.300000, l1_ratio=0.500000):
  RMSE: 0.8016125328127461
  MAE: 0.6212322644038427
  R2: 0.16903193835933472
Elasticnet model (alpha=0.300000, l1_ratio=0.300000):
  RMSE: 0.7945548693034976
  MAE: 0.619032025971276
  R2: 0.18359976351847052
Elasticnet model (alpha=0.800000, l1_ratio=0.500000):
  RMSE: 0.8584720333502169
  MAE: 0.647722377274032
  R2: 0.04696766019378218
Elasticnet model (alpha=0.450000, l1_ratio=0.300000):
  RMSE: 0.8035837903346776
  MAE: 0.6216927531522227
  R2: 0.16494002099488503
Elasticnet model (alpha=0.200000, l1_ratio=0.300000):
  RMSE: 0.7882632241950331
  MAE: 0.6164326257018996
  R2: 0.19647782690680782
Elasticnet model (alpha=0.900000, l1_ratio=0.900000):
  RMSE: 0.8600968020367704
  MAE: 0.6471596692693683
  R2: 0.043356773945086746


2022/05/03 20:16:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-model-ci, version 1
2022/05/03 20:16:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-model-ci, version 2


<ModelVersion: creation_timestamp=1651598192062, current_stage='Production', description='', last_updated_timestamp=1651598192068, name='sk-learn-model-ci', run_id='3cfe8a90187b40ebb9cd0d2bd472eb60', run_link='', source='./artifacts/1/3cfe8a90187b40ebb9cd0d2bd472eb60/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [3]:
client.delete_registered_model(reg_model_name)

### 1. Обзор существущей архитектуры и процесса ввода модели в эксплуатацию

В MLflow зарегистрировано несколько запусков эксперимента с различными показателями метрик.

Список экспериментов:

In [5]:
client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)
client.list_run_infos(experiment.experiment_id)

[<RunInfo: artifact_uri='./artifacts/1/4256cd4470f340429feae5957dcb75ef/artifacts', end_time=1651598191970, experiment_id='1', lifecycle_stage='active', run_id='4256cd4470f340429feae5957dcb75ef', run_uuid='4256cd4470f340429feae5957dcb75ef', start_time=1651598191153, status='FINISHED', user_id='sanityseeker'>,
 <RunInfo: artifact_uri='./artifacts/1/3cfe8a90187b40ebb9cd0d2bd472eb60/artifacts', end_time=1651598191146, experiment_id='1', lifecycle_stage='active', run_id='3cfe8a90187b40ebb9cd0d2bd472eb60', run_uuid='3cfe8a90187b40ebb9cd0d2bd472eb60', start_time=1651598190335, status='FINISHED', user_id='sanityseeker'>,
 <RunInfo: artifact_uri='./artifacts/1/294ea95825c54c3788c4c648977999d7/artifacts', end_time=1651598190329, experiment_id='1', lifecycle_stage='active', run_id='294ea95825c54c3788c4c648977999d7', run_uuid='294ea95825c54c3788c4c648977999d7', start_time=1651598189486, status='FINISHED', user_id='sanityseeker'>,
 <RunInfo: artifact_uri='./artifacts/1/810db23064cf46d8bfa880a65bf8

Одна из моделей зарегистрирована и введена в эксплуатацию.

Текущая модель выложенная в  эксплуатационную среду (переведенная в `Production`):

In [24]:
current_prod = [v for v in client.search_model_versions(f"name='{reg_model_name}'") if v.current_stage == 'Production'][-1]
current_prod

<ModelVersion: creation_timestamp=1651598192062, current_stage='Production', description='', last_updated_timestamp=1651598192068, name='sk-learn-model-ci', run_id='3cfe8a90187b40ebb9cd0d2bd472eb60', run_link='', source='./artifacts/1/3cfe8a90187b40ebb9cd0d2bd472eb60/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

Также есть новая версия этой модели, выложенная в тестовую среду (переведенная в `Staging`):

In [25]:
current_staging = [v for v in client.search_model_versions(f"name='{reg_model_name}'") if v.current_stage == 'Staging'][-1]
current_staging

<ModelVersion: creation_timestamp=1651598192027, current_stage='Staging', description='', last_updated_timestamp=1651598192034, name='sk-learn-model-ci', run_id='4256cd4470f340429feae5957dcb75ef', run_link='', source='./artifacts/1/4256cd4470f340429feae5957dcb75ef/artifacts/mdel', status='READY', status_message='', tags={}, user_id='', version='1'>

Задачей разработки в данном случае является обновление модели, не ухудшая метрик. Разберемся с этим детальнее. 

### 2. Процесс верификации новой версии и ввода в эксплуатацию

Прежде чем переводить текущую модель кандидата на ввод в эксплуатацию из тестовой среды в эксплуатационную, следует протестировать корректность модели.

Тестирование должно быть всесторонним и покрывать основные моменты эксплуатации модели.

Например, запустим веб-сервер с тестируемой моделью и попробуем отправить тестовые запросы:

In [18]:
! MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve -m "models:/sk-learn-model-ci/Staging" -p 5005

Traceback (most recent call last):
  File "/home/sanityseeker/anaconda3/envs/ml-env/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/sanityseeker/anaconda3/envs/ml-env/lib/pytho

Видно, что эта версия модели некорректна и ее не следует переводить в эксплуатацию, так как в этом случае сервер будет неработоспособен.

Обычно процесс верификации происходит с помощью систем непрерывной интеграции (например, Jenkins, TravisCI, CircleCI).

Эти системы как правило используют скрипты для принятия решений аналогичные рассмотренным далее.

### 3. Откат версии с тестовой среды и отметка модели

В случае, если тестирование модели неуспешно, необходимо откатить версию модели назад на стабильную и отметить не прошедшую тест модель, чтобы предотвратить ее возможную выкладку в будущем.

#### Откат версии на эксплуатационную в тестовой среде

В эксплуатационной среде сейчас находится работоспособная версия. Выложим эту же версию в тестовую среду, так как нам необходима работоспособная версия для последующего выбора кандидатов на ввод в эксплуатацию.

Выкладка стабильной версии с эксплуатационной среды (`Production`) в тестовую (`Staging`):

In [19]:
# Создание новой версии для тестовой среды из текущей эксплуатационной среды
result = client.create_model_version(
    name=current_prod.name,
    source=current_prod.source,
    run_id=current_prod.run_id
)
# Выкладка созданной версии в тестовую среду
client.transition_model_version_stage(
    name=current_prod.name,
    version=result.version,
    stage="Staging"
)

2022/05/03 02:13:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-model-ci, version 3


<ModelVersion: creation_timestamp=1651533181201, current_stage='Staging', description='', last_updated_timestamp=1651533181210, name='sk-learn-model-ci', run_id='29a3f9bd49b7449a8ad8d53ed7219a60', run_link='', source='./artifacts/1/29a3f9bd49b7449a8ad8d53ed7219a60/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

Такая операция также приводит тестовую среду к состоянию эксплуатационной, что позволяет проводить тестирование приближенное к эксплуатационной среде.

#### Отметка забракованной версии

В данном случае, ошибка была при регистрации модели - некорректно указан путь к файлу модели. 

Путь можно обновить на корректный:

In [26]:
new_staging = client.create_model_version(
    name=current_prod.name,
    source=current_staging.source.replace('mdel', 'model'), # баг был тут
    run_id=current_prod.run_id
)
client.transition_model_version_stage(
    name=current_prod.name,
    version=new_staging.version,
    stage="Staging"
)

2022/05/03 20:27:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-model-ci, version 3


<ModelVersion: creation_timestamp=1651598858504, current_stage='Staging', description='', last_updated_timestamp=1651598858524, name='sk-learn-model-ci', run_id='3cfe8a90187b40ebb9cd0d2bd472eb60', run_link='', source='./artifacts/1/4256cd4470f340429feae5957dcb75ef/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

Проверим, что теперь модель работает на тестовом сервере:

In [21]:
os.system('MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve -m "models:/sk-learn-model-ci/Staging" -p 5005 --no-conda &')

0

/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py:2322: FutureWarning: `--no-conda` is deprecated and will be removed in a future MLflow release. Use `--env-manager=local` instead.
  value = self.callback(ctx, self, value)
2022/05/03 02:13:29 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/05/03 02:13:29 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5005 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-05-03 02:13:29 +0300] [1020411] [INFO] Starting gunicorn 20.1.0
[2022-05-03 02:13:29 +0300] [1020411] [INFO] Listening at: http://127.0.0.1:5005 (1020411)
[2022-05-03 02:13:29 +0300] [1020411] [INFO] Using worker: sync
[2022-05-03 02:13:29 +0300] [1020413] [INFO] Booting worker with pid: 1020413


In [22]:
import requests

url = f'http://127.0.0.1:5005/invocations'

http_data = test_later_x[:10].to_json(orient='split')
response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')
print(test_later_y)

Predictions: [5.925164211632614, 5.964539037683882, 5.866425309090012, 5.983246407137424, 6.040856516155967, 5.476925826953726, 5.892059457193597, 5.826147448219497, 5.957386500329026, 5.733267307989582]
      quality
1876        6
147         4
3121        5
4778        6
4207        7
70          6
2870        7
1969        7
4289        6
1239        5


Чтобы предовратить в дальнейшем выкладку неработоспособной версии модели, можно отметить запуск подходящим тегом, например `staging: failed` и затем не использовать эту модель:

In [23]:
client.set_tag(current_staging.run_id, "staging", "failed")

Следующим образом можем сравнить метрики моделей:

In [24]:
new_staging_metrics = client.get_run(new_staging.run_id).data.metrics
prod_metrics = client.get_run(current_prod.run_id).data.metrics

# проверяем каждую метрику эксплуатационной модели
if all(new_staging_metrics[k] > v for k, v in prod_metrics.items()):
    # если новая модель лучше, то проставляем её как release candidate
    # и указываем, с какой версией сравнивалась
    client.set_tag(current_staging.run_id, "staging", "rc")
    client.set_tag(current_staging.run_id, "better_than", current_prod.version)

## Задание

In [8]:
def generate_submission():  # Генерация отчета выполнения задания
    client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
    
    runs = {}
    models = [
        {'name': m.name, 
         'versions': [
             {'current_stage': v.current_stage, 'run_id': v.run_id, 'status': v.status} 
             for v in m.latest_versions if m.name == 'sk-learn-model-ci']} 
        for m in client.search_registered_models()
    ]
    for e in client.list_experiments():
        if e.name == 'experiment-for-ci':
            for run_info in client.list_run_infos(e.experiment_id):
                run = mlflow.get_run(run_info.run_id)
                runs[run_info.run_id] = {'run_id': run_info.run_id, 'tags': run.data.tags, 'params': run.data.params, 'metrics': run.data.metrics}
    versions = [{'version': v.version, 'run_id': v.run_id} for v in client.search_model_versions(f"name='{reg_model_name}'")]
    with open('submission.json', 'w') as f:
        json.dump({'runs': runs, 'models': models, 'versions': versions}, f)


Процесс обновления модели состоит из двух этапов, первый - отбор кандидатов, которые лучше по общим метрикам, чем текущая модель в эксплуатации, второй - верификация модели на тестовом контуре также в сравнении с текущей эксплуатацией. Второй этап валидирует качество сервиса в целом, в то время как первый концентрируется именно на экспериментах с моделью.

В качестве задания предлагается автоматизировать шаги обработки моделей.

**1. Отметить следующими тегами все модели эксперимента (`run`), у которых метрика `rmse` лучше, чем у версии, которая сейчас используется в проде:**
  `staging` = `rc` (release candidate)
  
  с помощью `client.set_tag(run.info.run_id, "tag_key", "tag_value")`

  Если модель хуже, то следует проставить `staging` = `rejected`

  Также следует проставить тег
  `compared_with` = `%prod model version%` (возьмите версию из соответствующей модели)

In [102]:
from mlflow.tracking.client import MlflowClient

def get_run_metrics(client: MlflowClient, run_id: str) -> dict:
    return client.get_run(run_id).data.metrics

def tag_release_candidates(client: MlflowClient, model_name: str, exp_name: str, metric_name: str) -> None:
    curr_prod_model = [v for v in client.search_model_versions(f"name='{model_name}'") if v.current_stage == 'Production'][-1]
    reference_metric_value = get_run_metrics(client, curr_prod_model.run_id)[metric_name]
    
    exp_id = client.get_experiment_by_name(exp_name).experiment_id
    completed_runs = [run for run in client.search_runs(exp_id) if run.info.status == 'FINISHED']
    
    for run in completed_runs:
        run_metric_value = run.data.metrics[metric_name]
        
        if run_metric_value <= reference_metric_value:
            client.set_tag(run.info.run_id, 'staging', 'rc')
        else:
            client.set_tag(run.info.run_id, 'staging', 'rejected')
        
        client.set_tag(run.info.run_id, 'compared_with', curr_prod_model.version)

In [34]:
tag_release_candidates(client, reg_model_name, experiment_name, 'rmse')

**2. Для всех моделей, прошедших первичный отбор, в порядке ухудшениия значения метрики `RMSE` из эксперимента, выложите модель на стейджинг и запустите тестовые запросы.**

Посчитайте RMSE (напишите скрипт для запросов из `test_later_x`) ответов модели в тесте (`staging`) и в эксплуатации (`production`), если тестовые запросы дают показатели метрик хуже, чем у модели в эксплуатации, то модель следует пометить тегом `staging` = `failed`, затем перейти к следующей итерации - выложить на стейджинг следующего кандидата (`staging` = `rc`). Иначе следует выложить тестируемую модель в `Production` и обновить список кандидатов (тег `release candidates` из пункта 1).

In [35]:
def get_release_candidates(client: MlflowClient, exp_name: str, metric_name: str) -> list:
    exp_id = client.get_experiment_by_name(exp_name).experiment_id
    candidate_runs = [run for run in client.search_runs(exp_id) if run.data.tags['staging'] == 'rc']
    
    return sorted(candidate_runs, key=lambda run: run.data.metrics[metric_name], reverse=True)

In [36]:
rcs = get_release_candidates(client, experiment_name, 'rmse')

3. Повторять до тех пор, пока существуют кандидаты, проходящие первичный отбор.

Допишите скрипт, который итеративно выкладывает и верифицирует модели.

После выполнения скрипта, запустите `generate_submission()`, чтобы сгенерировать файл `submission.json` для отправки задания на проверку

In [21]:
def serve_model(model_name: str, stage_name: str, port: int):
    os.system(f'MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve -m "models:/{model_name}/{stage_name}" -p {port} --no-conda &')

In [78]:
import requests

def get_model_predictions(X_test: pd.DataFrame, port: int):
    url = f'http://127.0.0.1:{port}/invocations'

    http_data = X_test.to_json(orient='split')
    
    try:
        response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)
        if response.ok:
            return response.json()
    except Exception as e:
        raise ConnectionError(e)

In [27]:
serve_model(reg_model_name, 'Staging', 5005)

/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py:2322: FutureWarning: `--no-conda` is deprecated and will be removed in a future MLflow release. Use `--env-manager=local` instead.
  value = self.callback(ctx, self, value)
2022/05/03 20:28:02 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/05/03 20:28:02 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5005 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-05-03 20:28:02 +0300] [1063743] [INFO] Starting gunicorn 20.1.0
[2022-05-03 20:28:02 +0300] [1063743] [INFO] Listening at: http://127.0.0.1:5005 (1063743)
[2022-05-03 20:28:02 +0300] [1063743] [INFO] Using worker: sync
[2022-05-03 20:28:02 +0300] [1063745] [INFO] Booting worker with pid: 1063745


In [49]:
serve_model(reg_model_name, 'Production', 5006)

/home/sanityseeker/anaconda3/envs/ml-env/lib/python3.8/site-packages/click/core.py:2322: FutureWarning: `--no-conda` is deprecated and will be removed in a future MLflow release. Use `--env-manager=local` instead.
  value = self.callback(ctx, self, value)
2022/05/03 20:48:17 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/05/03 20:48:17 INFO mlflow.pyfunc.backend: === Running command 'gunicorn --timeout=60 -b 127.0.0.1:5006 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-05-03 20:48:17 +0300] [1064658] [INFO] Starting gunicorn 20.1.0
[2022-05-03 20:48:17 +0300] [1064658] [INFO] Listening at: http://127.0.0.1:5006 (1064658)
[2022-05-03 20:48:17 +0300] [1064658] [INFO] Using worker: sync
[2022-05-03 20:48:17 +0300] [1064660] [INFO] Booting worker with pid: 1064660


In [101]:
from tqdm.notebook import tqdm

def test_release_candidates(
    client: MlflowClient,
    model_name: str,
    candidates: list,
    X_test: pd.DataFrame,
    y_test: pd.DataFrame,
    error_func: callable,
    staging_port: int = 5005,
    prod_port: int = 5006,
) -> bool:
    prod_preds = get_model_predictions(X_test, prod_port)
    prod_metric_res = error_func(y_test, prod_preds)
    
    for cand in tqdm(candidates):
        model_version = client.create_model_version(
        name=model_name,
        source=f"{cand.info.artifact_uri}/model",
        run_id=cand.info.run_id
    )
        
        client.transition_model_version_stage(
        name=model_name,
        version=model_version.version,
        stage="Staging"
    )
        
        stage_preds = get_model_predictions(X_test, staging_port)
        stage_metric_res = error_func(y_test, stage_preds)
        
        if stage_metric_res < prod_metric_res:
            client.transition_model_version_stage(
                name=model_name,
                version=model_version.version,
                stage="Production"
            )
            print('Successfully deployed to Production!')
            return True  # return True if successful Staging deploy
        else:
            client.set_tag(cand.info.run_id, 'staging', 'rc')
    
    print('Nothing was deployed')
    return False  # return False if all candidates are worse than Prod

In [96]:
succeed = test_release_candidates(
    client,
    reg_model_name,
    rcs,
    test_later_x,
    test_later_y,
    error_func=lambda x, y: np.sqrt(mean_squared_error(x, y))
)

  0%|          | 0/35 [00:00<?, ?it/s]

2022/05/03 21:41:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-model-ci, version 6


In [99]:
if succeed:
    tag_release_candidates(client, reg_model_name, experiment_name, 'rmse')

**Итеративная выкладка лучших моделей**

In [ ]:
succeed = True

while succeed:

    tag_release_candidates(client, reg_model_name, experiment_name, 'rmse')
    rcs = get_release_candidates(client, experiment_name, 'rmse')

    succeed = test_release_candidates(
        client,
        reg_model_name,
        rcs,
        test_later_x,
        test_later_y,
        error_func=lambda x, y: np.sqrt(mean_squared_error(x, y))
    )

    if succeed:
        tag_release_candidates(client, reg_model_name, experiment_name, 'rmse')